## Import necessary libraries

In [66]:
import re
import requests
import json
from bs4 import BeautifulSoup
import pandas as pd
import time

## Set headers

In [67]:
# Depends on the browser being used. User agent can be retrieved from https://www.whatismybrowser.com
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/119.0.0.0 Safari/537.36 OPR/105.0.0.0"
}

## Scraping the data from understat.com

In [70]:
# Scrape single match shots
base_url = "https://understat.com/match/"

# List of match IDs
match_id_chunks = [list(range(start, min(start + 1000, 25000))) for start in range(0, 25000, 1000)]

# Initialize an empty DataFrame to store the data
df = pd.DataFrame()

# Start measuring overall execution time
start_time = time.time()

for match_ids_chunk in match_id_chunks:
    for match_id in match_ids_chunk:
        url = base_url + str(match_id)

        # Use requests to get the webpage and BeautifulSoup to parse the page
        response = requests.get(url, headers=headers)

        while True:
            if response.status_code == 404:
                print(f"{match_id} is giving 404 error")
                break
            elif response.status_code == 200:
                print(f"{match_id} status_code is 200")
                soup = BeautifulSoup(response.content, "html.parser")
                scripts = soup.find_all('script')

                # Get only the shotsData
                strings = scripts[1].string

                # Strip unnecessary symbols and get only JSON data
                str_start = strings.index("('")+2
                str_end = strings.index("')")
                json_data = strings[str_start:str_end]
                json_data = json_data.encode('utf8').decode('unicode_escape')

                # Convert string to JSON
                data = json.loads(json_data)

                # Seperate home and away teams into different DataFrames
                from pandas import json_normalize

                # data_all = json_normalize(data, sep = "_")
                data_home = json_normalize(data['h'])
                data_away = json_normalize(data['a'])
                match_df = pd.concat([data_home, data_away])

                # Concatenate the current DataFrame with the overall DataFrame
                df = pd.concat([df, match_df], ignore_index=True)

                break

# Overall execution time
end_time = time.time()

total_duration = end_time - start_time
print(f"Total execution time: {total_duration} seconds")

0 is giving 404 error
1 is giving 404 error
2 is giving 404 error
3 is giving 404 error
4 is giving 404 error
5 is giving 404 error
6 is giving 404 error
7 is giving 404 error
8 is giving 404 error
9 is giving 404 error
10 is giving 404 error
11 is giving 404 error
12 is giving 404 error
13 is giving 404 error
14 is giving 404 error
15 is giving 404 error
16 is giving 404 error
17 is giving 404 error
18 is giving 404 error
19 is giving 404 error
20 is giving 404 error
21 is giving 404 error
22 is giving 404 error
23 is giving 404 error
24 is giving 404 error
25 is giving 404 error
26 is giving 404 error
27 is giving 404 error
28 is giving 404 error
29 is giving 404 error
30 is giving 404 error
31 is giving 404 error
32 is giving 404 error
33 is giving 404 error
34 is giving 404 error
35 is giving 404 error
36 is giving 404 error
37 is giving 404 error
38 is giving 404 error
39 is giving 404 error
40 is giving 404 error
41 is giving 404 error
42 is giving 404 error
43 is giving 404 erro

In [71]:
df.head()

,id,minute,result,X,Y,xG,player,h_a,player_id,situation,season,shotType,match_id,h_team,a_team,h_goals,a_goals,date,player_assisted,lastAction
0,545909,27,BlockedShot,0.794000015258789,0.4209999847412109,0.10434672236442566,Juan Mata,h,554,DirectFreekick,2015,LeftFoot,81,Manchester United,Tottenham,1,0,2015-08-08 15:45:00,None,Standard
1,545910,27,BlockedShot,0.86,0.6270000076293946,0.064342200756073,Memphis Depay,h,555,SetPiece,2015,RightFoot,81,Manchester United,Tottenham,1,0,2015-08-08 15:45:00,Ashley Young,Pass
2,545911,35,BlockedShot,0.8430000305175781,0.33299999237060546,0.05715681612491608,Juan Mata,h,554,OpenPlay,2015,LeftFoot,81,Manchester United,Tottenham,1,0,2015-08-08 15:45:00,Wayne Rooney,Pass
3,545912,35,MissedShots,0.8480000305175781,0.5329999923706055,0.09214138984680176,Juan Mata,h,554,OpenPlay,2015,LeftFoot,81,Manchester United,Tottenham,1,0,2015-08-08 15:45:00,None,Tackle
4,545913,40,BlockedShot,0.8119999694824219,0.7069999694824218,0.03574201464653015,Memphis Depay,h,555,OpenPlay,2015,RightFoot,81,Manchester United,Tottenham,1,0,2015-08-08 15:45:00,Michael Carrick,BallRecovery


## Saving as a CSV

In [74]:
df.to_csv("shotsData.csv", index=False)